In [20]:
import numpy as np

n_population = 16000
population = np.zeros(n_population)
colours = ['white', 'red', 'blue', 'green', 'gray']
for group in range(1, len(colours)):
    population[:n_population//(2**group)] = group

for group in range(len(colours)):
    print("p(%s) = %.2f%%" % (colours[group], 100*np.sum(population==group)/population.shape[0]))

p(white) = 50.00%
p(red) = 25.00%
p(blue) = 12.50%
p(green) = 6.25%
p(gray) = 6.25%


In [ ]:
n_samples = 100
n_polls = 100